![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor


# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from itertools import combinations
from scipy.stats import ttest_ind
import statsmodels.api as sm
import scipy.stats as sst
from collections import Counter
import re

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

filenames_list = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        filenames_list.append(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import os
print(os.listdir("/kaggle/working"))

In [ ]:
cities_pop_filename = '/kaggle/input/world-cities/worldcities.csv'
cities_pop_filename

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Завернем модель в функцию для того, чтобы было удобнее вызывать

def model_func(df_preproc):
    # выделим тестовую часть
    train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
    test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

    y = train_data.Rating.values            # наш таргет
    X = train_data.drop(['Rating'], axis=1)
    
    RANDOM_SEED = 42
    
    # Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
    # выделим 20% данных на валидацию (параметр test_size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

    # Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
    model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
    
    # Обучаем модель на тестовом наборе данных
    model.fit(X_train, y_train)

    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
    # Предсказанные значения записываем в переменную y_pred
    y_pred = model.predict(X_test)
    
    result = metrics.mean_absolute_error(y_test, y_pred)
    
    # в RandomForestRegressor есть возможность вывести самые важные признаки для модели
    plt.rcParams['figure.figsize'] = (10,10)
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    feat_importances.nlargest(15).plot(kind='barh')
    
    plt.show
    
    return result

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
sample_submission

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

# Предобработка и анализ данных, создание новых признаков

## 1. Обработка NAN 

In [ ]:
# обработаем столбец Number of Reviews
number_rew_nan = pd.isna(data['Number of Reviews']).astype('uint8')

number_rew_nan.name = 'number_rew_nan'
number_rew_nan.value_counts()

In [ ]:
# Далее заполняем пропуски нулем

#mean = data['Number of Reviews'].mean()
number_rew = data['Number of Reviews'].fillna(0)

number_rew.name = 'number_rew'
number_rew.sample(5)

In [ ]:
# проверим модель на имеющихся числовых признаках без обработки

df_preproc = pd.concat([data.loc[:,['Rating', 'sample','Ranking']], number_rew, number_rew_nan], axis = 1)
model_func(df_preproc)

Отклонение предсказания - большое. Попробуем путем предобработки и создания новых признаков улучшить предсказание модели

In [ ]:
data.isna().sum()

Мы видим, что другие числовые столбцы не содержат пропусков, поэтому перейдем к следующему шагу.

## 2. Создадим новые числовые признаки

In [ ]:
data['Number of Reviews'].fillna(0, inplace = True)

In [ ]:
# посмотрим на топ 10 городов
for x in (data['City'].value_counts())[0:10].index:
    data['Ranking'][data['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение. 
Попробуем использовать этот факт для нормализации Ranking.

In [ ]:
data[data.City == 'London'].describe()

In [ ]:
data[(data.City == 'London') & (data.Rating == 5)].loc[:,['Number of Reviews', 'Ranking']].\
sort_values('Ranking', ascending = False).iloc[:10,]

In [ ]:
plt.figure(figsize = (5,5))
sns.jointplot(data = data[(data.City == 'London') & (data.Rating > 0)], x = 'Ranking', y = 'Rating', kind = 'kde')

Получается, что самые популярные рестораны (минимальный Ranking) имеют Rating равный примерно 4 баллам.
Но есть рестораны с высоким рейтингом 5, но по ранку расположенные примерно посередине. Это будет верно, например, для локальной забегаловки для местных, в которой не много посетителей, но которые все ставят ей стабильно 5.


In [ ]:
# создадим признак, который на основании Ranking вычисляет значение Rating. Зависимость линейная
# f(1) = 5 и f(n) = 1 при f(x) = k*x + b

max_rank_by_city = data.groupby(['City']).max().Ranking

def true_rating(row):
    return round(5 - 4*(1 - row['Ranking'])/(1 - max_rank_by_city[row['City']]),1)

data['rating_by_rank'] = data.apply(true_rating, axis = 1)
data['rating_by_rank'].sort_values()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (10,5))
ax = sns.heatmap(data.loc[data.City == 'London',['Rating', 'rating_by_rank']].corr(),annot = True, cmap = 'coolwarm')

In [ ]:
data['rating_by_rank'].hist()

Новый признак, построенный из Ranking путем лин. преобразования, тем не менее = равномерен и нормален.

In [ ]:
data[(data.City == 'London') & (data.Rating == 5)].loc[:,['Number of Reviews', 'Ranking', 'rating_by_rank']].\
sort_values('Ranking', ascending = False).iloc[:20,]

In [ ]:
df_preproc = pd.concat([data.loc[:,['Number of Reviews','rating_by_rank']], data.loc[:,['Rating', 'sample']]], axis = 1)
model_func(df_preproc)

После обработки Ranking предсказание модели значительно улучшилось.

In [ ]:
rating_by_rank = data['rating_by_rank']

In [ ]:
data[data.City == 'London'].loc[:,['Number of Reviews', 'Ranking', 'rating_by_rank','Rating']].\
sort_values('rating_by_rank', ascending = True).iloc[:20,]

In [ ]:
# создадим еще один признак на основе Ranking, проверим, какой работает лучше
# просто поделим Ranking на макс. значение в городе

def norm_rank_funk(row):
    return round(row['Ranking']*100/max_rank_by_city[row['City']],5)

norm_rank = data.apply(norm_rank_funk, axis = 1)
norm_rank.name = 'norm_rank'
norm_rank.sort_values()

In [ ]:
norm_rank.hist()

In [ ]:
plt.figure(figsize = (5,5))
plt.boxplot(norm_rank, vert = False)

Новый признак также не имеет смещений и выбросов.
Посмотрим на поведение модели на новом признаке

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample']], number_rew, number_rew_nan, norm_rank], axis = 1)
model_func(df_preproc)

Новый признак не особо повлиял на модель. Оставим для модели признак norm_rank.

In [ ]:
# среднее количество отзывов в городе
mean_rews_by_city = round((data.groupby(['City']).sum()['Number of Reviews']
                           /data.groupby(['City']).max()['Ranking']),2)

mean_rews = data.City.apply(lambda x: mean_rews_by_city[x])
mean_rews.name = 'mean_rews'
mean_rews.sample(5)

In [ ]:
# два признака для количества ресторанов в городе, один на основе макс Ranking, другой просто по количеству записей в городе

max_rank = data.City.apply(lambda x: max_rank_by_city[x])
max_rank.name = 'max_rank'

places_counts_by_sity = data.groupby(['City']).count().Ranking
places_counts = data.City.apply(lambda x: places_counts_by_sity[x])
places_counts.name = 'places_counts'

pd.concat([max_rank, places_counts], axis = 1).sample(10)

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample']], number_rew, number_rew_nan, norm_rank, mean_rews, places_counts], axis = 1)
model_func(df_preproc)

Модель улучшилась.

Посмотрим на Restaurant_id

In [ ]:
sns.countplot(data.Restaurant_id.value_counts()[:50])

В датасете встречаются рестораны с одинаковыми Restaurant_id. Судя по всему, эти рестораны пренадлежат одной сети. Сделаем новый признак.

In [ ]:
nets = data.Restaurant_id.value_counts()
rests_by_id = data.Restaurant_id.apply(lambda x: nets[x])
rests_by_id.name = 'rests_by_id'

In [ ]:
pd.concat([data.Restaurant_id,rests_by_id],axis = 1).sample(10)

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample']], number_rew, number_rew_nan, norm_rank, mean_rews, places_counts,\
                        max_rank, rests_by_id], axis = 1)
model_func(df_preproc)

Посмотрим на ID_TA

In [ ]:
print(data[data.ID_TA == 'd3161682'].iloc[0].URL_TA)
print(data[data.ID_TA == 'd3161682'].iloc[1].URL_TA)

In [ ]:
data.ID_TA.duplicated().value_counts()

В датасете есть рестораны с одинаковыми ID_TA, посмотрим на них

In [ ]:
data[data.ID_TA.duplicated()]

Все эти рестораны их Мадрида и Варшавы, что странно.

In [ ]:
ids = data[data.ID_TA.duplicated()].ID_TA.values
def is_dubl(item):
    if item in ids:
        return True
    else:
        return False

# посмотирм на пары этих дублей    
for id_ta in ids:
    display(data[data.ID_TA == id_ta])

Анализ пар показал, что это все-таки дубли. Прометим их и посмотрим, как ведет себя при этом модель

In [ ]:
# определим функцию для прометки дублей
def mark_dubl(row):
    id_ta = row.ID_TA
    ind = row.name
    if id_ta in ids:
        ind0 = data[data['ID_TA'] == id_ta].index.tolist()[0]
        ind1 = data[data['ID_TA'] == id_ta].index.tolist()[1]
        row0 = data[data['ID_TA'] == id_ta].iloc[0]
        row1 = data[data['ID_TA'] == id_ta].iloc[1]
        if (row0['sample'] == 0) and (row1['sample'] == 0):
            if row0.Ranking >= row1.Ranking:
                if ind0 == ind:
                    return 0
                else:
                    return 1
            else:
                if ind0 == ind:
                    return 1
                else:
                    return 0

        elif (row0['sample'] == 1) and (row1['sample'] == 0):
            if ind0 == ind:
                return 1
            else:
                return 0
            return 1
        elif (row0['sample'] == 0) and (row1['sample'] == 1):
            if ind0 == ind:
                return 0
            else:
                return 1
        elif (row0['sample'] == 1) and (row1['sample'] == 1):
            if row0.Ranking >= row1.Ranking:
                if ind0 == ind:
                    return 0
                else:
                    return 1
            else:
                if ind0 == ind:
                    return 1
                else:
                    return 0
    else:
        return 0
data['dubl'] = data.apply(mark_dubl, axis = 1)

In [ ]:
# проверим, как прометились дубли
for id_ta in ids:
    display(data[data.ID_TA == id_ta])

In [ ]:
data['dubl'].unique()

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'dubl']], number_rew, number_rew_nan, norm_rank, mean_rews, places_counts,\
                        rests_by_id], axis = 1)
model_func(df_preproc)

Предсказание немного улучшилось

Посмотрим на  URL_TA

In [ ]:
data.URL_TA.duplicated().value_counts()

Дубли мы уже прометили, посмотрим поближе на значения

In [ ]:
data[4:10]

Проанализируем значение ***gNNNNNN*** из url

In [ ]:
data['url_g'] = data.URL_TA.str.extract(r'(g\d+)')
data['url_g'] = data.url_g.apply(lambda x: int(x[1:]))
data['url_g'].sample(5)

In [ ]:
data['url_g'].value_counts()

In [ ]:
data['City'].value_counts()

In [ ]:
data[data['url_g'] == 6919449]

In [ ]:
data[data['url_g'].duplicated()]

Судя по всему, реквизит url_g содержит код населенного пункта на tadvisor

In [ ]:
len(data.url_g.unique())

In [ ]:
len(data.City.unique())

In [ ]:
cities = data.groupby(['City'])
cities['url_g'].value_counts()[:10]

In [ ]:
cities_gcnt = cities['url_g'].value_counts()
cities_gcnt['Brussels'][188644]

In [ ]:
def g_cnt(row):
    return cities_gcnt[row['City']][row['url_g']]

data['district_cnt'] = data.apply(g_cnt, axis = 1)


In [ ]:
data['district_cnt'].sample(3)

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan, norm_rank, mean_rews, places_counts,\
                        rests_by_id], axis = 1)
model_func(df_preproc)

С новым реквизитом предсказание немного улучшилось

### Склеим все новые признаки и посмотирм на матрицу корреляции

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (10,5))
ax = sns.heatmap(df_preproc.corr(),annot = True, cmap = 'coolwarm')

Очевидно, что признаки norm_rank и rating_by_rank имеют корреляцию = -1, так как получены один из другого путем линейного преобразования.

## 3. Обработка категориальных признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.info()

Посмотрим на признак City. Является ли он значимым для модели?

In [ ]:
all_cities = data.City.value_counts().index

fig, ax = plt.subplots(figsize = (15, 5))

sns.boxplot(x='City', y='Rating',data=data[data.Rating > 0],ax=ax)

plt.xticks(rotation=45)
ax.set_title('Boxplot for City')

plt.show()

Признак, очевидно, значим для определения Rating

In [ ]:
# выделим список с городами, боксплоты которых отличаются от остальных
phen_cities = [ i for i in all_cities if data[data.City == i].quantile(q = 0.75).Rating - data[data.City == i].quantile(q = 0.25).Rating != 1.5]
phen_cities  

### 3.1. Сконструируем dummy-признаки из City

In [ ]:
data.City.unique()

In [ ]:
# создадим отдельный признак с городами, у которых боксплоты отличаются от остальных

data['phen_cities'] = data.City.apply(lambda x: x if x in phen_cities else 'other')
data['phen_cities'].value_counts()

In [ ]:
# воспользуемся get_dummies

cities = pd.get_dummies(data.City, columns=[ 'City'])
cities.sample(5)

In [ ]:
phen_cities_dummy = pd.get_dummies(data.phen_cities, columns=[ 'phen_cities'])
phen_cities_dummy.sample(5)

In [ ]:
# проверим дамми признаки на модели

df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan,\
                        norm_rank, mean_rews, \
                        rests_by_id, cities], axis = 1)
model_func(df_preproc)

Модель дает более точное предсказание на всех городах. Оставим для модели cities

### 3.2. Обработаем признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

In [ ]:
data['Price Range'].isna().sum()

In [ ]:
# Пропусков в цене много. Выделим пропуски в цене в отдельный признак

price_isnan = pd.isna(data['Price Range']).astype('uint8')
price_isnan.name = 'price_isnan'

In [ ]:
# Определим функцию для заполнения Price Range

def price_ordinal(price):
    if price == '$':
        result = 1
    elif price == '$$ - $$$':
        result = 2
    elif price == '$$$$':
        result = 3
    else:
        result = 0
    return result

prices = data['Price Range'].apply(price_ordinal)
prices.name = 'prices'

prices.value_counts()

In [ ]:
# проверим модель с обработанной ценой

df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan,\
                        norm_rank, mean_rews, \
                        rests_by_id, cities,\
                        prices, price_isnan], axis = 1)
model_func(df_preproc)

### 3.3. Обработаем Cuisine Style

In [ ]:
# посмотрим, сколько всего стилей кухни встречается датасете

cuisine_styles = Counter()

for i in data['Cuisine Style'].dropna():
    l = re.sub('\s\'|\'','', i)[1:-1].split(',')
    cuisine_styles.update(l)

cuisines = [x[0] for x in cuisine_styles.most_common()]

len(cuisines)


In [ ]:
cuisine_styles.most_common()

In [ ]:
cuisines_groups = {
    'healty': ['Vegetarian Friendly', 'Gluten Free Options', 'Healthy', 'Vegan Options'],
    'taboo' : ['Halal', 'Kosher'],
    'seafood' : ['Seafood'],
    'alco' : ['Bar', 'Pub', 'Wine Bar', 'Gastropub', 'Brew Pub'],
    'fast' : ['Cafe', 'Fast Food', 'Diner', 'Street Food'],
    'other' : ['International', 'Fusion', 'Contemporary', 'Delicatessen'],
    'italian' : ['Italian', 'Pizza'],
    'eurapian' : ['European', 'Mediterranean', 'Italian', 'French', 'Spanish', 'British', 'Central European', 'Portuguese',\
           'German', 'Greek', 'Czech', 'Eastern European', 'Austrian', 'Polish', 'Scandinavian', 'Hungarian',\
           'Dutch', 'Irish', 'Belgian', 'Danish', 'Swiss', 'Swedish', 'Scottish', 'Norwegian',\
           'Slovenian', 'Russian', 'Croatian', 'Ukrainian', 'Romanian', 'Albanian', 'Welsh', 'Latvian'],
    'asian' : ['Asian', 'Japanese', 'Sushi', 'Chinese', 'Indian',  'Thai', 'Vietnamese', 'Korean', 'Pakistani', 'Nepali'\
         , 'Balti', 'Bangladeshi', 'Indonesian', 'Malaysian', 'Sri Lankan', 'Taiwanese', 'Tibetan',\
         'Cambodian', 'Singaporean', 'Mongolian', 'Filipino', 'Minority Chinese', 'Central Asian', 'Yunnan', \
         'Xinjiang'],
    'steak' : ['Steakhouse', 'Barbecue', 'Grill'],
    'eastern' : ['Middle Eastern', 'Turkish', 'Lebanese', 'Israeli', 'Persian', 'Arabic', 'Afghani', 'Uzbek'],
    'african' : ['African', 'Moroccan', 'Ethiopian', 'Egyptian', 'Tunisian'],
    'australian' : ['Australian', 'New Zealand'],
    'american' : ['American', 'Canadian'],
    'latamerican' : ['Mexican', 'South American', 'Latin', 'Argentinean', 'Central American', 'Brazilian', 'Peruvian',\
               'Venezuelan', 'Jamaican', 'Cuban', 'Colombian', 'Cajun & Creole', 'Southwestern',\
               'Chilean', 'Ecuadorean', 'Native American', 'Salvadoran'],
    'exotic' : ['Hawaiian', 'Polynesian', 'Jamaican', 'Cuban', 'Fujian', 'Burmese', 'Caribbean'],
    'сaucas' : ['Georgian', 'Armenian', 'Caucasian', 'Azerbaijani']
}

In [ ]:
cuisines_groups['asian']

In [ ]:
cuisine_most_common = [x[0] for x in cuisine_styles.most_common()[:10]]
cuisine_most_common

In [ ]:
# превратим Cuisine Style в список

cuisine_style = data['Cuisine Style'].apply(lambda x: ['other_style'] if pd.isnull(x) else x[1:-1].split(',') )
cuisine_style.sample(5)

In [ ]:
for i,k in enumerate(cuisine_style):
    new_list = []
    for j in k:
        j = re.sub('\s\'|\'','', j)
        new_list.append(j)
    cuisine_style.at[i] = new_list
cuisine_style.sample(5)

In [ ]:
# добавим новый признак "Количество кухонь в ресторане"

cuisine_counts = cuisine_style.apply(lambda x: len(x))
cuisine_counts.name = 'cuisine_counts'

cuisine_counts.sample(5)

In [ ]:
for col in cuisines_groups.keys():
    s1 = set(cuisines_groups[col])
    data[col] = cuisine_style.apply(lambda x: len(s1.intersection(set(x))))
    

In [ ]:
cuisines_cols = data.loc[:,cuisines_groups.keys()]
cuisines_cols.describe()

Изменим столбец Cuisine Style так: если стиль кухни ресторана попадает в самые частые значения, то оставляем его, если нет, меняем на other_style

In [ ]:
for i,k in enumerate(cuisine_style):
    new_list = []
    for j in k:
        if j in cuisine_most_common:
            new_list.append(j)
        else:
            new_list.append('other_style')
    cuisine_style.at[i] = new_list
cuisine_style.sample(5)

Теперь добавим новые признаки, соответсвующие самым частым значениям стилей кухонь

In [ ]:
cuisine_style_df = pd.DataFrame(cuisine_style)
for i in cuisine_most_common + ['other_style']:
    cuisine_style_df[i] = cuisine_style.apply(lambda x: 1 if i in x else 0).astype('uint8')

cuisine_style_df.drop('Cuisine Style', axis = 1, inplace=True)

cuisine_style_df.info()

In [ ]:
cuisine_style_df.sample(5)

In [ ]:
cuisines_cols.sample(5)

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan,\
                        norm_rank, mean_rews, rests_by_id, \
                        cities,
                        prices, price_isnan, \
                        cuisines_cols
                       ], axis = 1)
model_func(df_preproc)

In [ ]:
data['Cuisine Style'].isna().sum()

In [ ]:
data['Cuisine Style'].fillna('other_style', inplace = True)

### 3.4. Теперь обработаем Reviews
Выделим даты обзоров и посчитаем, сколько времени прошло между двумя обзорами

In [ ]:
data['rew_dates'] = data.Reviews.apply(lambda x : [0] if pd.isna(x) else x[2:-2].split('], [')[1][1:-1].split("', '"))
data['max_rew_date'] = pd.to_datetime(data['rew_dates'].apply(lambda x: max(x)))

data['first_rew'] = pd.to_datetime(data['rew_dates'].apply(lambda x : x[0]))
data['second_rew'] = pd.to_datetime(data['rew_dates'].apply(lambda x: x[1] if len(x) == 2 else ''))

rew_delta = np.abs(data['first_rew'] - data['second_rew'])
rew_delta = rew_delta.apply(lambda x: x.days)

rew_delta.name = 'rew_delta'

rew_delta.sample(5)

In [ ]:
rew_delta.isna().sum()

Пропусокв очень много, выделим отдельный признак для них

In [ ]:
# пустых значений много, сделаем новый признак для NAN
rew_delta_isnan = pd.isna(rew_delta).astype('uint8')

rew_delta_isnan.value_counts()

In [ ]:
# Заполним пропуски средним

mean = round(rew_delta.mean(), 2)
rew_delta = rew_delta.fillna(mean)
rew_delta.sample(5)

Создадим еще один признак для даты: количество дней, прошедшее между текущей датой и последним отзывом

In [ ]:
from datetime import datetime

rew_delta_cur = (datetime.now() - data['max_rew_date'])
rew_delta_cur = rew_delta_cur.fillna(rew_delta_cur.median())

rew_delta_cur = rew_delta_cur.apply(lambda x : x.days)

rew_delta_cur.name = 'rew_delta_cur'

rew_delta_cur.sample(5)

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan,\
                        norm_rank, mean_rews, rests_by_id, \
                        cities,
                        prices, price_isnan, \
                        cuisine_counts, cuisines_cols, \
                        rew_delta, rew_delta_cur,rew_delta_isnan
                       ], axis = 1)
model_func(df_preproc)

### 3.5. Сконструируем новый признак "Население города" на основе внешних данных

In [ ]:
data_add = pd.read_csv(cities_pop_filename)
data_add.sample(3)

In [ ]:
cities_info = pd.DataFrame(data.City.value_counts().index)
cities_info.columns = ['city']
cities_info.head(3)

In [ ]:
data_europe = data_add[data_add.iso2.apply(lambda x: x not in ('US','CA','VE'))]
data_europe.head()

In [ ]:
cities_country = cities_info.merge(data_europe, how = 'left', on = 'city').loc[:, ['city', 'iso2']]
cities_country.info()

In [ ]:
cities_country[cities_country.iso2.isna()]

In [ ]:
cities_country.at[23,'iso2'] = 'PT'
cities_country.at[25,'iso2'] = 'PL'
cities_country.at[22,'iso2'] = 'CH'
cities_country.at[19,'iso2'] = 'DK'
cities_country.info()

In [ ]:
cities_info = cities_info.merge(data_europe.loc[:,['city','capital', 'population']], how = 'left', on = 'city')
cities_info.sample(5)

In [ ]:
cities_info.isna().sum()

In [ ]:
cities_info['capital'] = cities_info.capital.fillna('not_cap')

In [ ]:
cities_info[cities_info.population.isna()]

In [ ]:
# заполним пропуски в населении и странах
cities_info.at[23,'population'] = 237591
cities_info.at[25,'population'] = 769498
cities_info.at[22,'population'] = 428737
cities_info.at[19,'population'] = 615993

In [ ]:
cities_info.columns =  ['City', 'capital', 'population']
cities_country.columns = ['City', 'country']

In [ ]:
# объединим с исходным датасетом

cities_pop = data.loc[:,['City']].merge(cities_info, how = 'left', on = 'City')

cities_pop.drop(['City'], axis = 1, inplace = True)

cities_pop.info()

In [ ]:
cities_capital = pd.get_dummies(cities_pop.capital)
cities_pop.drop(['capital'], axis = 1, inplace = True)

cities_capital.sample(5)

In [ ]:
# добавим дамми признаки для стран
countries = data.loc[:,['City']].merge(cities_country, how = 'left', on = 'City')

countries.drop(['City'], axis = 1, inplace = True)
countries.info()

In [ ]:
countries = pd.get_dummies(countries)
countries.info()

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan,\
                        norm_rank, mean_rews, rests_by_id, \
                        cities,
                        prices, price_isnan, \
                        cuisine_counts, cuisines_cols, \
                        rew_delta, rew_delta_cur,rew_delta_isnan,
                        cities_pop, cities_capital, countries
                       ], axis = 1)
model_func(df_preproc)

### 3.6. Количество туристов

In [ ]:
cities_info.info()

In [ ]:
cities_info['City']

In [ ]:
# заведем словарь для гоордов с новыми данными [кол-во млн. туристов, место в рейтенге благосостояния] по данным из wiki
th = {
    'London' : [19233, 14],
    'Paris' : [17560, 18],
    'Madrid' : [5440, 19],
    'Barcelona' : [6714, 19],
    'Berlin' : [5959, 15],
    'Milan' : [6481, 24],
    'Rome' : [10065, 24],
    'Prague' : [8949, 22],
    'Lisbon' : [3539, 29],
    'Vienna' : [6410, 2],
    'Amsterdam' : [8354, 7],
    'Brussels' : [3942, 13],
    'Hamburg' : [1450, 15],
    'Munich' : [4067, 15],
    'Lyon' : [6000, 18],
    'Stockholm' : [2605, 8],
    'Budapest' : [3823, 31],
    'Warsaw' : [2850, 27],
    'Dublin' : [5213, 16],
    'Copenhagen' : [3070, 5],
    'Athens' : [5728, 36],
    'Edinburgh' : [1660, 14],
    'Zurich' : [2240, 6],
    'Oporto' : [2341, 29],
    'Geneva' : [1150, 6],
    'Krakow' : [2732, 27],
    'Oslo' : [1400, 1],
    'Helsinki' : [1240, 9],
    'Bratislava' : [126, 26],
    'Luxembourg' : [1139, 11],
    'Ljubljana' : [5900, 20]
}

In [ ]:
tourists = data.City.apply(lambda x : th[x][0])
tourists.name = 'tourists'

hapiness = data.City.apply(lambda x : th[x][1])
hapiness.name = 'hapiness'

tourists
hapiness

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt']], number_rew, number_rew_nan,\
                        norm_rank, mean_rews, rests_by_id, \
                        cities,
                        prices, price_isnan, \
                        cuisine_counts, cuisines_cols, \
                        rew_delta, rew_delta_cur,rew_delta_isnan,
                        cities_pop, cities_capital, countries,
                        tourists, hapiness
                       ], axis = 1)
model_func(df_preproc)

### 3.7. Построим на основе текстов обзоров новые признаки

In [ ]:
# выделим текст обзоров для последующего анализа.
data['rew_texts'] = data.Reviews.apply(lambda x : '' if pd.isna(x) else x[2:-2].split('], [')[0])

rew_texts_list = data['rew_texts'].apply(lambda x : [''] if x == '' else x.split("', '"))

data['first_text'] = rew_texts_list.apply(lambda x : x[0][1:-1] if len(x) == 1 else x[0][1:] if len(x) == 2 else '')
data['second_text'] = rew_texts_list.apply(lambda x: x[1][:-1] if len(x) == 2 else '')

data.loc[:,['Reviews', 'rew_texts', 'first_text', 'second_text']].sample(5)

In [ ]:
def rew_counts_func(row):
    result = 0 if row['rew_texts'] == ''  else 1 if row['second_text'] == '' else 2
    return result
    
rew_counts = data.apply(rew_counts_func, axis = 1)
rew_counts.name = 'rew_counts'

pd.concat([rew_counts, data['rew_texts']], axis = 1)

In [ ]:
rew_counts.value_counts()

In [ ]:
# количество слов в отзывах

words_count = data['rew_texts'].apply(lambda x : len(x.split()))

words_count.name = 'words_count'
pd.concat([rew_counts, data['rew_texts'], words_count], axis = 1)

In [ ]:
all_words = Counter()

for i in (data['first_text'].str.split() + data['second_text'].str.split()):
    all_words.update(i)

words = [x[0] for x in all_words.most_common()]

all_words.most_common()

In [ ]:
positive = ['fine', 'better', 'great', 'good', 'nice', 'excellent', 'best', 'lovely', 'delicious',\
            'amazing', 'friendly', 'atmosphere', 'tasty',  'perfect', 'wonderful', 'super', 'top','cosy',\
            'beautiful', 'pleasant', 'brilliant', 'fantastic', 'cool', 'outstanding','favorite',\
            'enjoyable', 'welcome', 'incredible', 'awesome', 'charming', 'original'
           ]
negative = ['worth','bad', 'poor', 'terrible', 'slow', 'worst','disappointing', 'overpriced', 'awful',\
            'rude','horrible', 'too'
           ]

In [ ]:
def format_rew(rew):
    sent = rew.split()
    new_sent = []
    for word in sent:
        if re.search('\w+', word.lower()) is None:
            continue
        else:
            new_word = re.search('\w+', word.lower()).group(0)
            new_sent.append(new_word)
    return new_sent

In [ ]:
data['rew_texts1'] = data.rew_texts.apply(format_rew)
data.loc[:,['rew_texts','rew_texts1']].sample(5)

In [ ]:
def tone_pos(item):
    cnt = 0
    for word in positive:
        if word in item:
            cnt +=1
    return cnt

def tone_neg(item):
    cnt = 0
    for word in negative:
        if word in item:
            cnt +=1
    return cnt

In [ ]:
data['rew_pos'] = data.rew_texts1.apply(tone_pos)
data['rew_neg'] = data.rew_texts1.apply(tone_neg)

In [ ]:
data.loc[:,['rew_texts1', 'rew_pos', 'rew_neg']].sample(10)

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt','rew_pos', 'rew_neg']], \
                        number_rew, number_rew_nan,\
                        norm_rank, mean_rews, rests_by_id, \
                        cities,
                        prices, price_isnan, \
                        cuisine_counts, , \
                        rew_delta, rew_delta_cur,rew_cuisines_colsdelta_isnan,
                        cities_pop, cities_capital, countries,
                        tourists, hapiness,
                        rew_counts
                       ], axis = 1)
model_func(df_preproc)

Количество слов модель ухудшило. А количество отзывов - наоборот. Оставим признак rew_counts

### 3.8. На основе признаков population, tourists и Number of Reviews построим новые признаки

In [ ]:
# посмотрим на корреляцию признаков

X = pd.concat([number_rew, cities_pop, tourists, data['Rating']], axis = 1)

fig, ax = plt.subplots(1,1, figsize = (10,5))
ax = sns.heatmap(X.corr(),annot = True, cmap = 'coolwarm')

Количество туристов и население города очень сильно положительно скоррелированы.
Попробуем использовать главную компоненту вместо этого признака.
Сначала нормируем признаки.

# Final model test

In [ ]:
df_preproc = pd.concat([data.loc[:,['Rating', 'sample', 'district_cnt','rew_pos', 'rew_neg']], \
                        number_rew, number_rew_nan,\
                        norm_rank, mean_rews, rests_by_id, \
                        cities,
                        prices, price_isnan, \
                        cuisine_counts, cuisines_cols, \
                        rew_delta, rew_delta_cur,rew_delta_isnan,
                        cities_pop, cities_capital, countries,
                        tourists, hapiness,
                        rew_counts
                       ], axis = 1)
model_func(df_preproc)

# Submission
готовим Submission на кагл

In [ ]:
# выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values           # наш таргет
X = train_data.drop(['Rating'], axis=1)
    
RANDOM_SEED = 42
    
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators= 200, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
    
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

y_pred = np.round(y_pred * 2) / 2
print('MAE: ',metrics.mean_absolute_error(y_test, y_pred))
    
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)
sample_submission.Rating

Обратила внимание, что Rating в датасете - заполняет значения от 1 до 5 с шагом 0.5. Поэтому, если округлить предсказание до ближайшеего x.5 числа, то предсказание улучшится.

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission = np.round(predict_submission * 2)/2


In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission